# Crimes in Boston Analysis 

Project from Kaggle Challenge : https://www.kaggle.com/datasets/AnalyzeBoston/crimes-in-boston

In [2]:
import sqlite3

In [3]:
import numpy as np
import pandas as pd

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import statsmodels.formula.api as smf

# Analysis for the mayor's team

During the last municipal campaign in Boston, criminality has been a major topic of debates. As citizens have expressed strong expectations from her on that front, the newly-elected mayor of Boston is looking for data-based insights on criminality in the Massachussetts capital. She has mandated your economics and urbanism consulting firm, The Locomotive, for this study.

## Load Database

In [13]:
# Download database
!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/boston_crimes.sqlite > db/boston_crimes.sqlite

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19.1M  100 19.1M    0     0  11.2M      0  0:00:01  0:00:01 --:--:-- 11.2M


## Explore the database

Three tables are available :

- the incident_reports table has been provided by the Police Department of Boston. Each observation corresponds to a criminal incident that has required an intervention by the police in the municipality of Boston;
- the districts table has been provided by the Urbanism Department of Boston. It gathers geographical information about the various police districts of Boston;
- and the indicators table has been shared by the Economics Department of Boston, which keeps track of various indicators of the social and economic activity of Boston neighborhoods. Each observation corresponds to a police district.

In [40]:
# ==> I'm using DBeaver to analyse these three tables**

In [41]:
# ==> A schema of the database is available in the db folder**

## Extract dataset

We want to investigate the influence of the socio-economic characteristics of Boston's different districts on the number of crime reports and incidents. To do so, we need to extract the relevant dataset. Each row should correspond to one of the 12 police districts of Boston (as listed in the districts table of the database).

To identify the district, we will need the following columns:

- the CODE of the police district (1 letter and 1 or 2 numbers);
- the full NAME of the police district.
- NB_INCIDENTS, i.e. the total number of incidents reported in the police district over the period covered by the data at hand (2015-2019)
- several socio-economic indicators: 
MEDIAN_AGE;
TOTAL_POP;
PERC_OF_30_34;
PERC_MARRIED_COUPLE_FAMILY;
PER_CAPITA_INCOME;
PERC_OTHER_STATE_OR_ABROAD;
PERC_LESS_THAN_HIGH_SCHOOL;
PERC_COLLEGE_GRADUATES.

In [16]:
# SQL query to build the dataset
query = """
    SELECT
        districts.CODE,
        districts.NAME, 
        COUNT(incident_reports.INCIDENT_NUMBER) AS NB_INCIDENTS,
        indicators.MEDIAN_AGE,
        indicators.TOTAL_POP,
        indicators.PERC_OF_30_34,
        indicators.PERC_MARRIED_COUPLE_FAMILY,
        indicators.PER_CAPITA_INCOME,
        indicators.PERC_OTHER_STATE_OR_ABROAD,
        indicators.PERC_LESS_THAN_HIGH_SCHOOL,
        indicators.PERC_COLLEGE_GRADUATES
    FROM districts
    JOIN incident_reports ON incident_reports.DISTRICT = districts.CODE 
    JOIN indicators ON indicators.CODE = districts.CODE 
    GROUP BY districts.CODE
    ORDER BY NB_INCIDENTS DESC
"""

In [17]:
db_path = 'db/boston_crimes.sqlite'
conn = sqlite3.connect(db_path)
c = conn.cursor()

In [20]:
crimes_df = pd.read_sql_query(query, conn)
crimes_df

,CODE,NAME,NB_INCIDENTS,MEDIAN_AGE,TOTAL_POP,PERC_OF_30_34,PERC_MARRIED_COUPLE_FAMILY,PER_CAPITA_INCOME,PERC_OTHER_STATE_OR_ABROAD,PERC_LESS_THAN_HIGH_SCHOOL,PERC_COLLEGE_GRADUATES
0,B2,Roxbury,38877,32.5,54161,27.8,17.8,20978,2.9,23.0,18.9
1,C11,Dorchester,32875,33.4,126909,28.2,26.6,29767,2.4,18.0,17.1
2,D4,South End,31258,37.1,32571,33.9,28.3,83609,6.2,11.8,8.5
3,B3,Mattapan,28331,36.7,26659,20.9,29.8,28356,2.3,14.5,22.9
4,A1,Downtown,26260,33.5,18306,32.5,35.8,80057,14.8,15.4,6.9
5,C6,South Boston,16617,31.9,36772,46.1,24.7,64745,2.4,7.9,8.4
6,D14,Brighton,13788,30.8,55297,52.8,26.4,41261,8.6,6.7,10.5
7,E13,Jamaica Plain,12802,34.8,40867,32.5,33.7,51655,5.5,8.0,12.1
8,E18,Hyde Park,12551,39.4,38924,21.1,38.4,32744,1.9,13.8,21.3
9,A7,East Boston,9691,30.6,47263,31.1,30.4,31473,3.5,27.2,11.5


In [21]:
# check of the shape
crimes_df.shape

(12, 11)

In [23]:
crimes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CODE                        12 non-null     object 
 1   NAME                        12 non-null     object 
 2   NB_INCIDENTS                12 non-null     int64  
 3   MEDIAN_AGE                  12 non-null     float64
 4   TOTAL_POP                   12 non-null     int64  
 5   PERC_OF_30_34               12 non-null     float64
 6   PERC_MARRIED_COUPLE_FAMILY  12 non-null     float64
 7   PER_CAPITA_INCOME           12 non-null     int64  
 8   PERC_OTHER_STATE_OR_ABROAD  12 non-null     float64
 9   PERC_LESS_THAN_HIGH_SCHOOL  12 non-null     float64
 10  PERC_COLLEGE_GRADUATES      12 non-null     float64
dtypes: float64(6), int64(3), object(2)
memory usage: 1.2+ KB


## Linear Regression

We want to investigate the impact of the socio-economic characteristics of the different Boston police districts on the number of incidents that are reported in these areas.

We are going to use the number of incidents as dependent variable
our regressors will be the various socio-economic indicators extracted from the database.

In [24]:
crimes_df.columns

Index(['CODE', 'NAME', 'NB_INCIDENTS', 'MEDIAN_AGE', 'TOTAL_POP',
       'PERC_OF_30_34', 'PERC_MARRIED_COUPLE_FAMILY', 'PER_CAPITA_INCOME',
       'PERC_OTHER_STATE_OR_ABROAD', 'PERC_LESS_THAN_HIGH_SCHOOL',
       'PERC_COLLEGE_GRADUATES'],
      dtype='object')

In [28]:
model = smf.ols(formula='NB_INCIDENTS ~ MEDIAN_AGE + PERC_OF_30_34 + PERC_MARRIED_COUPLE_FAMILY + PERC_OTHER_STATE_OR_ABROAD \
+ PERC_LESS_THAN_HIGH_SCHOOL + PERC_COLLEGE_GRADUATES', data=crimes_df)
model = model.fit()
model.summary()

/home/ameliermd/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           NB_INCIDENTS   R-squared:                       0.899
Model:                            OLS   Adj. R-squared:                  0.777
Method:                 Least Squares   F-statistic:                     7.391
Date:                Wed, 11 May 2022   Prob (F-statistic):             0.0222
Time:                        15:00:50   Log-Likelihood:                -114.59
No. Observations:                  12   AIC:                             243.2
Df Residuals:                       5   BIC:                             246.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                   1.358e+04   6.16e+04      0.220      0.834   -1.45e+05    1.72e+05
MEDIAN_AGE                  2252.7344   1237.522      1.820      0.128    -928.416    5433.885
PERC_OF_30_34               -532.9973    468.940     -1.137      0.307   -1738.447     672.452
PERC_MARRIED_COUPLE_FAMILY -2115.1839    408.344     -5.180      0.004   -3164.866   -1065.502
PERC_OTHER_STATE_OR_ABROAD  1787.6939    532.432      3.358      0.020     419.034    3156.354
PERC_LESS_THAN_HIGH_SCHOOL   112.7375    493.977      0.228      0.829   -1157.072    1382.547
PERC_COLLEGE_GRADUATES        19.4699    458.775      0.042      0.968   -1159.850    1198.790
==============================================================================
Omnibus:                        6.933   Durbin-Watson:                   2.325
Prob(Omnibus):                  0.031   Jarque-Bera (JB):                3.155
Skew:                           1.148   Prob(JB):                        0.206
Kurtosis:                       4.020   Cond. No.                     2.42e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.42e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Analysis of MEDIAN_AGE


In [29]:
# median age is associated with a positive effect (see the positive coef in the summary)

In [31]:
# for median age, t_statistic = 1.820 (from summary)
# t_statistic = coef / standard error

In [32]:
t_statistic = 2252.7344 / 1237.522
t_statistic

1.8203590724043692

In [34]:
# recompute t_stat with the summary's data
coef = model.params['MEDIAN_AGE']
std_err = model.bse['MEDIAN_AGE']
t_stat = coef / std_err
t_stat

1.820359754327443

In [35]:
# p_value of MEDIAN_AGE
p_value = 0.128
#     P value is the probability that what you observed is just due to pure chance
#     La proba que l'hypothèse nulle soit vraie est ici de 12.8% : cela reste assez élevé. 
#     La corrélation entre le median age et le nb of incidents n'est donc pas complètement avérée.

### Regressors whose effect is statistically significant

In [36]:
# The regressors whose effect is statistically significant at the 95% confidence level in this regression model are :
# 'PERC_MARRIED_COUPLE_FAMILY' and 'PERC_OTHER_STATE_OR_ABROAD' (smallest p_value)

In [38]:
# For 1% increases of married couples in the district, the number of incidents decreases of -2115 
# (see coef of PERC_MARRIED_COUPLE_FAMILY)

### Limits of this regression model

In [42]:
# Comme observé précédemment, les p values de 4 regressors sont élevées : median_age, perc of 30-34, 
# high_school et college_graduates. 
# Seuls married_couple et state_or_abroad ont le plus d'effet sur le nb of incidents.
# D'autre part, nous n'avons que 12 lignes, donc 12 observations : cela est très faible pour construire une regression.

# Analysis for the police department

## Data manipulation

In [51]:
# Load csv
!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/incident_reports.csv > data/incident_reports.csv   
!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/districts.csv > data/districts.csv    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19.1M  100 19.1M    0     0  7884k      0  0:00:02  0:00:02 --:--:-- 7887k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   498  100   498    0     0   2515      0 --:--:-- --:--:-- --:--:--  2515


In [52]:
# Load dataset
from data import load_data_viz_data
data = load_data_viz_data()
data.head()

,INCIDENT_NUMBER,OFFENSE_CODE_GROUP,SHOOTING,OCCURRED_ON_DATE,LAT,LONG,NAME,LAT_POLICE_STATION,LONG_POLICE_STATION
0,I192068249,Other,0,2015-08-28 10:20:00,42.330119,-71.084251,Roxbury,42.328894,-71.085359
1,I182074094,Violence and harassment,0,2015-09-14 09:31:00,42.315142,-71.067047,Roxbury,42.328894,-71.085359
2,I182054888,Violence and harassment,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359
3,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359
4,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359


In [49]:
data.shape

(237221, 9)

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237221 entries, 0 to 237220
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   INCIDENT_NUMBER      237221 non-null  object 
 1   OFFENSE_CODE_GROUP   237221 non-null  object 
 2   SHOOTING             237221 non-null  int64  
 3   OCCURRED_ON_DATE     237221 non-null  object 
 4   LAT                  237221 non-null  float64
 5   LONG                 237221 non-null  float64
 6   NAME                 237221 non-null  object 
 7   LAT_POLICE_STATION   237221 non-null  float64
 8   LONG_POLICE_STATION  237221 non-null  float64
dtypes: float64(4), int64(1), object(4)
memory usage: 18.1+ MB


### Most represented type of incident per district

We want a dataframe with : 
- one row per police district, designated by its full name
- one text column that indicates the name of the most common category of incident in the district over the whole sample period.

In [58]:
most_incident = pd.DataFrame(data.groupby('NAME')['OFFENSE_CODE_GROUP'].agg(pd.Series.mode))
most_incident
# pd.Series.mode -> Return the mode(s) of the Series. The mode is the value that appears most often.

,OFFENSE_CODE_GROUP
NAME,
Brighton,Larceny and vandalism
Charlestown,Larceny and vandalism
Dorchester,Larceny and vandalism
Downtown,Larceny and vandalism
East Boston,Larceny and vandalism
Hyde Park,Larceny and vandalism
Jamaica Plain,Larceny and vandalism
Mattapan,Police investigation procedure
Roxbury,Larceny and vandalism


In [72]:
# Sorted list of all incidents in Brighton district
brighton_incidents = data[data['NAME'] == 'Brighton'].groupby('OFFENSE_CODE_GROUP').count().sort_values('INCIDENT_NUMBER', ascending=False)
brighton_incidents[['INCIDENT_NUMBER']]

,INCIDENT_NUMBER
OFFENSE_CODE_GROUP,
Larceny and vandalism,5274
Police investigation procedure,2783
Fraud and law violations,1807
Violence and harassment,1627
Other,925
Drugs and disorderly conduct,809
Disputes,563


### Average distance to the police station per district

In [73]:
# Haversine distance function
from math import radians, sin, cos, asin, sqrt

def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Compute distance (km) between two pairs of (lat, lng) coordinates
    See - (https://en.wikipedia.org/wiki/Haversine_formula)
    """
    
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    
    return 2 * 6371 * asin(sqrt(a))

In [92]:
# New column with the average distance between incident & police station per district
distance_df = pd.DataFrame(data)
distance_df['average_distance'] = distance_df.apply(lambda x: haversine_distance(x.LONG, x.LAT, x.LONG_POLICE_STATION, \
                                                                                 x.LAT_POLICE_STATION), axis=1)
distance_df

,INCIDENT_NUMBER,OFFENSE_CODE_GROUP,SHOOTING,OCCURRED_ON_DATE,LAT,LONG,NAME,LAT_POLICE_STATION,LONG_POLICE_STATION,average_distance
0,I192068249,Other,0,2015-08-28 10:20:00,42.330119,-71.084251,Roxbury,42.328894,-71.085359,0.163832
1,I182074094,Violence and harassment,0,2015-09-14 09:31:00,42.315142,-71.067047,Roxbury,42.328894,-71.085359,2.145919
2,I182054888,Violence and harassment,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359,2.021175
3,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359,2.021175
4,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359,2.021175
...,...,...,...,...,...,...,...,...,...,...
237216,I172049757,Police investigation procedure,0,2018-08-09 18:00:00,42.355216,-71.060129,Charlestown,42.361800,-71.060300,0.732215
237217,I172049757,Drugs and disorderly conduct,0,2018-08-09 18:00:00,42.355216,-71.060129,Charlestown,42.361800,-71.060300,0.732215
237218,I172027369,Other,0,2018-12-12 05:00:00,42.376852,-71.068873,Charlestown,42.361800,-71.060300,1.815839
237219,I172027369,Police investigation procedure,0,2018-12-12 05:00:00,42.376852,-71.068873,Charlestown,42.361800,-71.060300,1.815839


In [93]:
# DataFrame with one row per police district, designated by its full name, and one column that displays this average Haversine distance
distance_df = pd.DataFrame(distance_df[['NAME', 'average_distance']].groupby(by='NAME')['average_distance'].mean())
distance_df

,average_distance
NAME,
Brighton,3.770867
Charlestown,58.758394
Dorchester,3.174444
Downtown,6.336657
East Boston,5.568626
Hyde Park,2.397613
Jamaica Plain,2.585737
Mattapan,1.908636
Roxbury,2.989029


In [95]:
distance_df.sort_values(by='average_distance', ascending=False, inplace=True)
distance_df

,average_distance
NAME,
Charlestown,58.758394
South Boston,31.454220
West Roxbury,18.370045
Downtown,6.336657
East Boston,5.568626
Brighton,3.770867
Dorchester,3.174444
Roxbury,2.989029
Jamaica Plain,2.585737


In [101]:
# Distance for Brihghton district
distance_df[distance_df.index == 'Brighton']

,average_distance
NAME,
Brighton,3.770867


## Data visualisation

See the second Notebook 'boston_crimes_dataviz.ipynb'

## Short presentation

See the pdf presentation for the head of the Police Department